In [1]:
import numpy as np 
import pandas as pd
from collections import defaultdict


In [2]:
usecols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [3]:
df_train = pd.read_csv('input/train_ver2.csv',usecols=usecols)
sample = pd.read_csv('input/sample_submission.csv')


In [4]:
df_train = df_train.drop_duplicates(['ncodpers'], keep='last')

df_train.fillna(0, inplace=True)

In [5]:
df_train.tail()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
13647304,1166765,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647305,1166764,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647306,1166763,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647307,1166789,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647308,1550586,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
pro_col = usecols[1:]

In [7]:
product_list = df_train[pro_col].sum(axis=0).tolist()

In [8]:
id_preds = {}
for row in sample.values:
    id = row[0]
    id_preds[id] = product_list

In [9]:
# check if customer already have each product or not. 
already_active = {}
for row in df_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(df_train.columns[1:], row) if c[1] > 0]
    already_active[id] = active

# add 7 products(that user don't have yet), higher probability first -> train_pred   
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(df_train.columns[1:], p) if i[0] not in already_active[id]],
                                  key=lambda i:i [1], 
                                  reverse=True)[:7]]
    train_preds[id] = preds
    
test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))
    
sample['added_products'] = test_preds

In [10]:
sample.to_csv('output/probableRecent2.csv', index=False)

In [11]:
df_train[pro_col].sum(axis=0).sort_values()

ind_aval_fin_ult1        16.0
ind_ahor_fin_ult1        78.0
ind_cder_fin_ult1       316.0
ind_deco_fin_ult1       318.0
ind_deme_fin_ult1      1021.0
ind_pres_fin_ult1      2244.0
ind_viv_fin_ult1       2961.0
ind_hip_fin_ult1       4560.0
ind_plan_fin_ult1      7367.0
ind_ctju_fin_ult1      7602.0
ind_ctma_fin_ult1      8103.0
ind_fond_fin_ult1     14713.0
ind_valo_fin_ult1     21426.0
ind_dela_fin_ult1     31220.0
ind_ctpp_fin_ult1     33284.0
ind_tjcr_fin_ult1     34843.0
ind_reca_fin_ult1     47555.0
ind_nomina_ult1       48374.0
ind_nom_pens_ult1     52923.0
ind_cno_fin_ult1      73110.0
ind_ecue_fin_ult1     75946.0
ind_ctop_fin_ult1    100658.0
ind_recibo_ult1      113602.0
ind_cco_fin_ult1     562259.0
dtype: float64